## 第8章：Planning and Learning with Tabular Methods （表格规划与学习法）

第8章探讨了强化学习中"规划"（planning）和"学习"（learning）之间的关系，重点介绍如何在表格方法（tabular methods）下结合两者的优势。这一章将前几章的无模型学习（如TD和蒙特卡洛）与基于模型的规划（如动态规划）统一起来，主要提出了**Dyna**架构作为关键创新。

但实际上于今日（2025年）的强化学习中，"规划"和"学习"的界限已经模糊，而且框架也不再局限于表格方法。现代强化学习（如深度强化学习）通常使用函数逼近（neural network）来处理大规模状态空间。

### 8.1 背景与核心问题
- **学习与规划的区别**：
  - **学习（Learning）**：从与环境的真实交互中获取经验，更新价值估计或策略（如第6章的TD学习、第7章的n步方法）。
  - **规划（Planning）**：利用环境模型（model），通过计算或模拟来改进策略（如第4章的动态规划）。

- **挑战**：
  - 无模型方法（如Q-learning）直接从经验学习，效率可能较低，尤其在稀疏奖励环境中。
  - 基于模型的方法（如动态规划）需要完整模型，但现实中模型通常未知。

- **目标**：
  - 结合学习和规划的优势：***用经验学习模型，再用模型进行规划***。

### 8.2 环境模型（Models）
- **定义**：
  - 环境模型是对状态转移和奖励的描述，通常包括：
    - 状态转移函数：$P(s'|s, a)$（给定状态和动作，下一个状态的概率分布）。
    - 奖励函数：$R(s, a, s')$（转移到新状态时的期望奖励）。

- **类型**：
  - **分布模型（Distribution Models）**：给出完整的概率分布，如马尔可夫决策过程（MDP）的转移概率。
  - **采样模型（Sample Models）**：通过模拟生成状态和奖励的样本（如模拟器）。

- **作用**：
  - 模型允许智能体"想象"未来的状态和奖励，而无需真实交互。

### 8.3 规划与动态规划的联系
- **动态规划回顾**（第4章）：
  - 价值迭代：$V(s) \leftarrow \max_a \sum_{s'} P(s'|s, a) [R(s, a, s') + \gamma V(s')]$
  - 策略迭代：评估策略后改进策略。

- **规划的本质**：
  - 使用模型进行价值更新或策略优化，与动态规划类似，但可以基于部分经验或模拟数据。

- **局限**：
  - 动态规划假设模型已知，而第8章关注如何从经验中学习模型并用于规划。

### 8.4 集成规划与学习的Dyna架构
- **Dyna概念**：
  - Dyna（Dynamic Programming + Learning）是一种将直接学习（real experience）和规划（simulated experience）结合的框架。
  - 核心思想：智能体不仅从真实经验中学习，还通过模型生成模拟经验进行额外更新。

- **Dyna-Q算法**：
  - Dyna-Q结合Q-learning（无模型学习）和模型-based规划。
  - **步骤**：
    1. **真实经验**：执行动作 $a_t$，观察 $s_{t+1}$ 和 $r_{t+1}$，更新Q值：
       $$Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha [r_{t+1} + \gamma \max_a Q(s_{t+1}, a) - Q(s_t, a_t)]$$
    2. **模型学习**：记录转移 $(s_t, a_t) \to (s_{t+1}, r_{t+1})$ 到表格模型中。
    3. **规划**：从已访问的状态-动作对中随机采样 $(s, a)$，用模型生成 $(s', r)$，更新：
       $$Q(s, a) \leftarrow Q(s, a) + \alpha [r + \gamma \max_{a'} Q(s', a') - Q(s, a)]$$
    - 规划步骤重复n次（n为规划步数）。

- **伪代码**：
  ```
  初始化 Q(s, a) 和 Model(s, a) 为任意值
  重复：
      s ← 当前状态
      选择动作 a（基于Q，如ε-贪婪）
      执行 a，观察 r 和 s'
      Q(s, a) ← Q(s, a) + α [r + γ max_a' Q(s', a') - Q(s, a)]  # 真实经验更新
      Model(s, a) ← (s', r)  # 更新模型
      重复 n 次（规划）：
          从已访问的 (s, a) 中随机采样
          从 Model(s, a) 获取 s' 和 r
          Q(s, a) ← Q(s, a) + α [r + γ max_a' Q(s', a') - Q(s, a)]
      s ← s'
  ```

- **优点**：
  - 比纯Q-learning更快收敛，因为规划利用了模型生成的额外经验。
  - 比纯动态规划更实际，因为模型是从经验中学习的。

### 8.5 Dyna-Q的示例
- **迷宫任务**：
  - 书中用一个简单迷宫展示了Dyna-Q的效率。
  - Q-learning只依赖真实步数找到目标，而Dyna-Q通过规划提前"预演"路径，所需真实交互更少。

### 8.6 规划中的优先级（Prioritized Sweeping）
- **问题**：
  - Dyna-Q随机采样状态-动作对进行规划，效率可能不高。

- **优先级扫描（Prioritized Sweeping）**：
  - 根据更新的大小（即TD误差 $|r + \gamma \max_a Q(s', a) - Q(s, a)|$）给状态-动作对分配优先级。
  - 优先更新可能带来更大价值变化的状态。

- **伪代码片段**：
  ```
  维护优先级队列 PQueue
  每次真实经验后：
      计算TD误差，加入队列
      重复n次：
          从队列中取出优先级最高的 (s, a)
          用模型更新 Q(s, a)
          检查受影响的状态，更新队列
  ```

- **效果**：
  - 在稀疏奖励任务中显著提高效率。

### 8.7 实时动态规划（Real-Time Dynamic Programming, RTDP）
- **定义**：
  - RTDP是一种基于真实轨迹的规划方法，仅更新智能体实际访问的状态。
  - 更新规则与值迭代类似，但只在当前状态上执行。

- **特点**：
  - 结合了在线学习和规划，适用于实时任务。

### 8.8 与前几章的联系
- **第6章（TD学习）**：
  - TD和Q-learning是无模型方法，Dyna通过引入模型增强了这些算法。

- **第7章（n步自举）**：
  - n步方法关注多步回报，而Dyna关注如何利用模型模拟多步经验。

- **第4章（动态规划）**：
  - Dyna将动态规划的思想（模型-based更新）引入经验驱动的学习。

### 8.9 理论与实践
- **收敛性**：
  - 在适当条件下（如充分探索），Dyna-Q和优先级扫描收敛到最优策略。

- **局限**：
  - 表格方法假设状态和动作空间有限，下一章（第9章）将探讨函数逼近解决大规模问题。

- **应用**：
  - Dyna架构启发了现代模型基强化学习（如MuZero）。

## 总结
第8章通过Dyna架构整合了规划与学习，展示了如何用经验构建模型并利用模拟经验加速学习。Dyna-Q和优先级扫描等方法在表格设置下提高了效率，为后续基于模型的强化学习（如第9章的函数逼近）奠定了基础。这一章的核心思想在现代RL（如AlphaGo的蒙特卡洛树搜索）中仍有深远影响。


In [1]:
"""
以下是一个简化的Dyna-Q实现（伪迷宫环境），其中Q值逐渐反映从起点到目标的最优路径。
实现了实际交互、模型更新和规划：
"""
import numpy as np

# 简单迷宫：3x3网格，起点(0,0)，目标(2,2)，障碍(1,1)
n_rows, n_cols = 3, 3
n_states = n_rows * n_cols
n_actions = 4  # 上、右、下、左
Q = np.zeros((n_states, n_actions))
model = {}  # (s, a) -> (s', r)
alpha, gamma, epsilon = 0.1, 0.9, 0.1
n_planning = 5

def state_to_coords(s): return divmod(s, n_cols)
def coords_to_state(r, c): return r * n_cols + c
def step(s, a):
    r, c = state_to_coords(s)
    new_r, new_c = r, c
    if a == 0: new_r = max(0, r-1)  # 上
    elif a == 1: new_c = min(n_cols-1, c+1)  # 右
    elif a == 2: new_r = min(n_rows-1, r+1)  # 下
    elif a == 3: new_c = max(0, c-1)  # 左

    # 检查是否撞到障碍物
    if new_r == 1 and new_c == 1:  # 障碍物位置(1,1)
        new_r, new_c = r, c  # 撞到障碍物后不移动

    s_next = coords_to_state(new_r, new_c)
    reward = 1 if s_next == 8 else 0  # 目标(2,2)
    return s_next, reward

# Dyna-Q训练
for episode in range(100):
    s = 0  # 起点(0,0)
    while s != 8:  # 直到到达目标
        # 选择动作
        if np.random.rand() < epsilon:
            a = np.random.randint(n_actions)
        else:
            a = np.argmax(Q[s])
        # 真实经验
        s_next, r = step(s, a)
        Q[s, a] += alpha * (r + gamma * np.max(Q[s_next]) - Q[s, a])
        model[(s, a)] = (s_next, r)
        # 规划
        for _ in range(n_planning):
            s_sim, a_sim = list(model.keys())[np.random.randint(len(model))]
            s_next_sim, r_sim = model[(s_sim, a_sim)]
            Q[s_sim, a_sim] += alpha * (r_sim + gamma * np.max(Q[s_next_sim]) - Q[s_sim, a_sim])
        s = s_next
print("Q-table:\n", Q)

Q-table:
 [[0.65608026 0.7289999  0.59031339 0.65605928]
 [0.7289933  0.80999999 0.72898023 0.65607906]
 [0.80999266 0.80997643 0.9        0.72898351]
 [0.65607076 0.         0.         0.59021391]
 [0.         0.         0.         0.        ]
 [0.80999529 0.89999495 1.         0.89999092]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]


In [2]:
"""
# 可视化Q值
表格的每一行代表一个状态（3x3网格共9个状态）。
每一列代表一个动作（0上、1右、2下、3左）。Q值越高，表示该状态-动作对越优。
换句话说，在每一个状态下（每一行），选择列索引最大的动作（Q值最大的动作）就是最优策略。
→ → ↓
↑ ↑ ↓
↑ ↑ ↑
"""
print("Optimal policy（最佳策略）:\n", np.argmax(Q, axis=1).reshape(n_rows, n_cols))

Optimal policy（最佳策略）:
 [[1 1 2]
 [0 0 2]
 [0 0 0]]
